# Named Volumes 

In [ ]:
cd notebooks/docker-code/docker-volumes-deep/

In [ ]:
ls

In [ ]:
!docker image ls

### Buildiamo la nuova immagine

In [ ]:
!docker build . -t docker-volumes-0:named

In [ ]:
# Tralasciando le rige npm, cosa noti di particolare negli step di build?

### Creazione del nuovo container con un Named Volume

In [ ]:
!docker image ls

In [ ]:
#Dovresti già conoscere il comando usato per eseguire il nuovo container, prova ad indovinare
!docker run --name volumes-deep -p 8080:80 -v named:/app/feedback --rm docker-volumes-0:named

In [ ]:
#Come risolvere il problema?
!docker ps
!docker stop f0088d8dc37e
!docker rm f0088d8dc37e # dovrebbe già essere cancellato perchè lanciato con --rm

In [ ]:
!docker run --name volumes-deep -p 8080:80 -v named:/app/feedback --rm -d docker-volumes-0:named

### Verifica presenza del volume

In [ ]:
!docker ps 

In [ ]:
!docker inspect f21ca64f902c | grep volume

In [ ]:
!docker volume ls

In [ ]:
!docker stop f21ca64f902c

In [ ]:
!docker rm f21ca64f902c # se lanciato senza --rm

### Riavvio container

In [ ]:
!docker run --name volumes-deep -p 8080:80 -v named:/app/feedback --rm -d docker-volumes-0:named

In [ ]:
# Apri il browser
# Verifica presenza del feedback precedentemente lasciato

In [ ]:
!docker stop 1c8c092bfb64e

# Anonymous Volumes

In [ ]:
!docker volume ls

In [ ]:
!docker volume inspect feedback

In [ ]:
!docker volume inspect named

### Creiamo un container senza volumi

In [ ]:
!docker run --name volumes-deep-2 -p 8081:80 --rm -d docker-volumes-0:named

In [ ]:
!docker ps

In [ ]:
!docker volume ls

In [ ]:
#In realtà un nuovo volume è già stato creato
#Questo perchè all'interno del Dockerfile è stato inserito il comando VOLUME

In [ ]:
!cat Dockerfile

[Modifichiamo il Dockerfile](/tree/notebooks/docker-code/docker-volumes-deep)

### Buildiamo la nuova immagine

In [ ]:
!docker build . -t docker-volumes-0:anonymous

### Stoppiamo e cancelliamo il container creato

In [ ]:
!docker stop d5cd57762a48645e50a9aee82680d1fcdd57caeeea00ba4437776165faaa6614

### Lanciamo un nuovo comando senza volume

In [ ]:
!docker run --name volumes-deep-3 -p 8081:80 --rm -d docker-volumes-0:anonymous

In [ ]:
### Verifichiamo il numero di volumi
### Possiamo vedere che nessun nuovo volume è stato creato
!docker volume ls

In [ ]:
!docker stop 09110e8968b0f6fd395964d6cb60dd76edf4b43a962e18181511e68bce6d04ba

### Gli Anonymous Volumes possono essere creati anche non specificando il nome del volume dal comando Docker

In [ ]:
!docker run --name volumes-deep -p 8080:80 -v /app/feedback --rm -d docker-volumes-0:anonymous

In [ ]:
#Possiamo vedere che l'anonymous volume è stato creato
!docker volume ls

In [ ]:
!docker stop 403f299df63fc9cdf8ab89c58f333702e8c8dd2496a85a9adbfe9f7bb538461f

In [ ]:
!docker volume ls

# Binding Mount

In [ ]:
!docker run --name volumes-deep -p 8080:80 -v "global-path":/app/feedback --rm -d docker-volumes-0:anonymous

In [ ]:
#Parleremo più nel dettaglio di questi container più avanti

## Continuare con la parte di networking nel prossimo notebook